In [1]:
#we may need some code in the ../python directory and/or matplotlib styles
import sys
sys.path.append('../python/')

#matplotlib for plotting
import matplotlib as mpl
from matplotlib import pyplot as plt
plt.style.use('../mplstyles/stylelib/standard.mplstyle')

#other computational libraries
import numpy as np
import pandas as pd
import scipy.constants as co
import scipy.stats as ss
import itertools
import pickle
from scipy import signal

In [2]:
import ENDF6

In [7]:
#get some cross section data
f = open('/Users/villaa/Misc/ENDF/ENDF-B-VIII.0_neutrons/n-014_Si_028.endf')
lines = f.readlines()
sec = ENDF6.find_section(lines, MF=4, MT=2)  # capture cross-section
#x, y = ENDF6.read_table(sec)

In [8]:
for i in sec:
    print(i)

 1.402800+4 2.773700+1          0          3          0          01425 4  2

 0.000000+0 2.773700+1          0          2          0          01425 4  2

 0.000000+0 0.000000+0          0          0          1        4131425 4  2

        413          2                                            1425 4  2

 0.000000+0 1.000000-5          0          0          2          01425 4  2

 0.000000+0 0.000000+0                                            1425 4  2

 0.000000+0 4.000000+4          0          0          4          01425 4  2

 2.935700-2 3.573400-4 4.154000-5 0.000000+0                      1425 4  2

 0.000000+0 5.200000+4          0          0          4          01425 4  2

 5.718200-2 1.386800-3 6.333900-5 0.000000+0                      1425 4  2

 0.000000+0 5.400000+4          0          0          4          01425 4  2

-2.120300-3 2.946100-3 2.498200-5 0.000000+0                      1425 4  2

 0.000000+0 1.300000+5          0          0          4          01425 4  2


In [10]:
print(ENDF6.list_content(sec))

{(1425, 4, 2)}
